# First Aid Kit

It's a pity, but it is impossible to develop applications that work perfectly.
Applications always contain errors due to programming mistakes, incorrect use of function calls, or external reasons, such as a missing file or the abnormal termination of an external (non-APL) piece of code.

All these problems may interrupt the normal processing of an application, and require immediate corrective action.
The aim of this chapter is to give you the basis to deal with these situations, and work through the experience.

In [a later chapter](./Event-Handling.ipynb) you will see that it is possible to trap and hide unpredictable errors or events, in order to provide a safe environment for the user.

Because APL is an interpreted language, program execution is not aborted, just suspended when an exception occurs.
Then the programmer has access to all the application variables, and can diagnose the problem.
If they can modify the code or correct some environmental dependency, they may be able to restart the program from the very point it had been interrupted in, until a new exception occurs, and so on.
This gives great flexibility when debugging applications.

In this chapter you will learn how to:
 - interpret error messages;
 - interpret the information provided by various system indicators;
 - trace the execution of an application step by step; and
 - set *breakpoints* in strategic places to help debugging.
 
A forgotten comma, an incorrect axis, a missing parenthesis, a scalar where you expected a vector, can all cause an error and it may sometimes be very difficult for a beginner to understand the reason.
Try to get help from people who have a better understanding of APL:
you will save time *and* learn a lot from them.